In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



url = "https://raw.githubusercontent.com/CharlesCLuo/Application-of-AI-in-Supply-Chain-Risk-Management-Series/main/Demand_Forecsting/comprehensive_supply_chain_data_with_weather_spike.csv"
df = pd.read_csv(url, parse_dates=['Timestamp'], index_col='Timestamp')



features = ['Inventory Levels', 'Lead Time (days)', 'Supplier Reliability (%)',
            'Economic Indicator (GDP Growth %)', 'Marketing Spend ($)',
            'Price Fluctuation (%)', 'Competitor Pricing ($)', 'Customer Sentiment',
            'Transportation Costs ($)', 'Stockouts (Binary)', 'Promotions and Discounts (%)',
            'Product Quality (%)', 'Temperature (°C)']

X = df[features]
y = df['Demand']




X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.05,
    'max_depth': 4,
    'n_estimators': 300,
    'subsample': 0.7,
    'colsample_bytree': 0.9,
    'random_state': 42
}

model = xgb.XGBRegressor(**params)
model.fit(X_train_scaled, y_train)



temperature_scenarios = [0, 10, 25, 35, 45]  # Freezing, Cold, Normal, Hot, Heatwave
scenario_predictions = []

X_median = X.median()

for temp in temperature_scenarios:
    X_scenario = X_median.copy()
    X_scenario['Temperature (°C)'] = temp
    X_scenario_scaled = scaler.transform([X_scenario])
    predicted_demand = model.predict(X_scenario_scaled)[0]
    scenario_predictions.append(predicted_demand)

plt.figure(figsize=(10, 6))
plt.bar(['Freezing (0°C)', 'Cold (10°C)', 'Normal (25°C)', 'Hot (35°C)', 'Heatwave (45°C)'],
        scenario_predictions, color=['blue', 'green', 'orange', 'purple', 'red'])
plt.xlabel('Temperature Scenarios')
plt.ylabel('Predicted Demand')
plt.title('Figure 8.15. Predicted Demand Under Different Temperature Scenarios')
plt.show()
